# Plateforme Agnostique de Traitement et d'Analyse des Textes
### Carnet d'expérimentation
---

## Sujet : Import et publication recueil csv

---

# Mise en forme données
---

In [2]:
cd ../..

/Volumes/Geek/Work/Patat


In [3]:
import importlib
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
tqdm.pandas()

## GoogleSheet Recueil

In [4]:
import patat.util.gsheet

In [5]:
gs = patat.util.gsheet.GoogleSheet()

In [6]:
sheet_id = '1RFxwIBYqa_eYcx_LZpTdhi47QbLcesz9NByMfGzdJ8s'
range_name = 'articles labellisés'

In [7]:
df_brut = gs.read_df(sheet_id,range_name)

In [8]:
df_brut

,#site,# article,Site,url,Auteur/trice,Date,Titre,Explication (pour les infox seulement),infox,signé,...,inversion de la preuve,invérifiable,titre décalé,dénigrement,cherry picking,degré infox,observateur,thème (abandonné),type infox (abandonné),Date de collection (abandonné)
0,1,1,Alnas,https://www.alnas.fr/actualite/communaute/lara...,,31/10/2022,L’Arabie saoudite organise des célébrations d'...,,non,non,...,non,non,non,non,non,,FB,None,None,None
1,1,2,Alnas,https://www.alnas.fr/actualite/en-vrac/quel-ar...,,19/10/2022,« Quel art ! » : un étudiant en droit invente ...,,non,non,...,non,non,non,non,non,,FB,None,None,None
2,1,3,Alnas,https://www.alnas.fr/actualite/politique/laust...,,18/10/2022,L’Australie ne reconnaît plus Jérusalem comme ...,,non,non,...,non,non,non,non,non,,FB,None,None,None
3,1,4,Alnas,https://www.alnas.fr/actualite/communaute/nant...,,17/10/2022,Nantes : une mère musulmane meurt poignardée e...,,non,non,...,non,non,non,non,non,,FB,None,None,None
4,1,5,Alnas,https://www.alnas.fr/actualite/en-vrac/la-fran...,,10/11/2022,La France condamnée pour avoir placé un enfant...,le titre est décalé (donne une impression faus...,non,non,...,non,non,oui,non,non,,FB,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,44,618,Wikistrike,https://www.wikistrike.com/2022/10/la-pologne-...,,31/10/2022,La Pologne s’apprête à s’emparer du territoire...,relai de ragots invérifiés,oui,non,...,non,oui,oui,non,non,2,FB,None,None,None
619,44,619,Wikistrike,https://www.wikistrike.com/2022/11/exclusivite...,,19/11/2022,Exclusivité WikiStrike: un physicien en colère...,délire total d'un physicien anonyme (ahurissan...,oui,non,...,non,non,non,non,non,1,FB,None,None,None
620,44,620,Wikistrike,https://www.wikistrike.com/2022/12/zelensky-pa...,,23/12/2023,Zelensky parti demander à papa Biden une aide ...,l'article est vide (même si le titre accuse Ze...,oui,non,...,non,non,non,non,non,1,FB,None,None,None
621,44,621,Wikistrike,https://www.wikistrike.com/2023/01/le-chef-du-...,,03/01/2023,Le chef du renseignement iranien assure que la...,long compte rendu d'un interview du chef du re...,non,non,...,non,non,non,non,non,,FB,None,None,None


In [9]:
import urllib
def get_site(url):
    parse = urllib.parse.urlparse(url)
    if parse:
        return parse.netloc

In [10]:
df_brut['site']=df_brut['url'].apply(get_site)

In [11]:
df_brut[df_brut['site']=='']

,#site,# article,Site,url,Auteur/trice,Date,Titre,Explication (pour les infox seulement),infox,signé,...,invérifiable,titre décalé,dénigrement,cherry picking,degré infox,observateur,thème (abandonné),type infox (abandonné),Date de collection (abandonné),site


In [12]:
df_brut.value_counts('site')

site
www.francesoir.fr                    117
www.breizh-info.com                   50
lesmoutonsrebelles.com                44
www.wikistrike.com                    44
lezarceleurs.blogspot.com             36
lesdeqodeurs.fr                       28
lemediaen442.fr                       23
www.profession-gendarme.com           19
www.dreuz.info                        19
fr.sott.net                           18
www.polemia.com                       17
www.mondialisation.ca                 17
www.nouvelordremondial.cc             14
reseauinternational.net               14
www.lelibrepenseur.org                14
www.voltairenet.org                   12
www.lesalonbeige.fr                   10
www.anguillesousroche.com              9
lecourrier-du-soir.com                 9
www.preuvesduparanormal.fr             8
lesobservateurs.ch                     8
www.les-crises.fr                      8
lesakerfrancophone.fr                  7
www.alnas.fr                           7
www.medias-

## Normalisation labels

In [13]:
from patat.imp.labels import label_translator
from patat.imp.labels import short_label_names

In [14]:
df_recueil = df_brut.rename(columns=label_translator)
df_label = df_recueil[['url'] + short_label_names].copy()

In [15]:
import numpy as np
def normalize_label(value):
    if type(value) != str:
        return np.NaN
    elif value == 'oui':
        return 1.0
    elif value =='non' :
        return 0.0
    else:
        return np.NaN

In [16]:
for label in short_label_names:
    df_label[label]=df_label[label].apply(normalize_label)

In [17]:
df_label

,url,infox,signe,entites_nommees,entites_coherentes,ouverture_esprit,qualite_ecriture,faits,opinions,fausse_nouvelle,...,trop_mots,trop_chiffres,sophisme,inversion_preuve,inverifiable,propos_raportes,sources_citees,titre_decale,denigrement,cherry_picking
0,https://www.alnas.fr/actualite/communaute/lara...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,https://www.alnas.fr/actualite/en-vrac/quel-ar...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,https://www.alnas.fr/actualite/politique/laust...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,https://www.alnas.fr/actualite/communaute/nant...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,https://www.alnas.fr/actualite/en-vrac/la-fran...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,https://www.wikistrike.com/2022/10/la-pologne-...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0
619,https://www.wikistrike.com/2022/11/exclusivite...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
620,https://www.wikistrike.com/2022/12/zelensky-pa...,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
621,https://www.wikistrike.com/2023/01/le-chef-du-...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


## Importation et parsing articles

In [18]:
from patat.imp.importer import Importer
from patat.imp.parser import Parser

In [19]:
imp = Importer()
parser = Parser()

### Chargement html

In [20]:
pages = []
for url in df_recueil['url']:
    print(url)
    try:
        html = imp.get_webpage(url)
        pages.append({
            'url' : url,
            'html' : html,
        })
    except:
        print(f'Error : {url}')

https://www.alnas.fr/actualite/communaute/larabie-saoudite-organise-des-celebrations-dhalloween-autrefois-interdites/
https://www.alnas.fr/actualite/en-vrac/quel-art-un-etudiant-en-droit-invente-un-systeme-ingenieux-pour-tricher-pendant-ses-examens/
https://www.alnas.fr/actualite/politique/laustralie-ne-reconnait-plus-jerusalem-comme-capitale-disrael/
https://www.alnas.fr/actualite/communaute/nantes-une-mere-musulmane-meurt-poignardee-en-pleine-rue-le-suspect-en-fuite/
https://www.alnas.fr/actualite/en-vrac/la-france-condamnee-pour-avoir-place-un-enfant-musulman-dans-une-famille-chretienne/
https://www.alnas.fr/actualite/communaute/hassan-iquioussen-la-justice-belge-refuse-de-remettre-limam-a-la-france/
https://www.alnas.fr/actualite/en-vrac/les-emirats-arabes-unis-et-larabie-saoudite-fournissent-764-des-besoins-du-japon-en-petrole/
https://www.anguillesousroche.com/actualite/ouverture-du-gazoduc-baltic-pipe-qui-relie-la-norvege-et-la-pologne/
https://www.anguillesousroche.com/technolo

https://lesmoutonsrebelles.com/quest-ce-que-le-grand-reset-et-que-veulent-reellement-les-globalistes/
https://lezarceleurs.blogspot.com/2022/05/le-dr-zelenko-raconte-comment-le-msm.html
https://lezarceleurs.blogspot.com/2022/05/enquete-5g-sur-les-sabotages-dantennes.html
https://lezarceleurs.blogspot.com/2022/06/augmentation-de-2000-des-lesions.html
https://lezarceleurs.blogspot.com/2022/07/les-convulsions-post-vaccinales-des.html
https://lezarceleurs.blogspot.com/2022/07/enfants-vaccines-de-8100-30200-plus-de.html
https://lezarceleurs.blogspot.com/2022/07/vaccins-covid-19-et-escrocs-associes.html
https://lezarceleurs.blogspot.com/2022/07/mais-quont-ils-fait-les-peuples-pour.html
https://lezarceleurs.blogspot.com/2022/07/leurope-occidentale-na-pas-besoin-de.html
https://lezarceleurs.blogspot.com/2022/07/ukraine-les-buchers-de-la-haine-par.html
https://lezarceleurs.blogspot.com/2022/07/des-senateurs-refusent-des-informations.html
https://lezarceleurs.blogspot.com/2022/07/reduction-de-la

ERROR:root:Importer.get_webpage:https://planetes360.fr/urgent-votre-argent-les-interesse-un-reset-qui-pourrait-bien-ruiner-des-millions-de-francais/


https://planetes360.fr/sarah-ransome-victime-depstein-affirme-posseder-une-video-identifiant-des-elites-pedocriminelles/
https://planetes360.fr/la-france-vit-tous-les-symptomes-dun-effondrement-depuis-plusieurs-annees-maintenant-et-malheureusement-cela-risque-de-sempirer-et-la-chute-risque-de-saccelerer/


ERROR:root:Importer.get_webpage:https://planetes360.fr/la-france-vit-tous-les-symptomes-dun-effondrement-depuis-plusieurs-annees-maintenant-et-malheureusement-cela-risque-de-sempirer-et-la-chute-risque-de-saccelerer/


https://www.polemia.com/la-droite-face-a-limperatif-culturel/
https://www.polemia.com/elisabeth-borne-et-lecole-cynisme-et-deconstruction/
https://www.polemia.com/jean-francois-carenco-a-loutre-mer-mensonge-et-repression-au-programme/
https://www.polemia.com/le-defile-du-14-juillet-une-histoire-du-lien-national/
https://www.polemia.com/une-guerre-civile-menee-contre-les-peuples-indigenes-deurope/
https://www.polemia.com/les-j-o-2024-en-seine-saint-denis-une-folie-criminelle/
https://www.polemia.com/pouvoir-dachat-la-grande-arnaque-par-jean-yves-le-gallou/
https://www.polemia.com/pap-ndiaye-a-leducation-nationale-deconstruction-et-hypocrisie/
https://www.polemia.com/italie-vers-un-prochain-blocus-a-limmigration-clandestine/
https://www.polemia.com/disparition-de-francois-bernard-huyghe-un-phare-en-pleine-tempete/
https://www.polemia.com/callac-laboratoire-du-peuplement-de-la-france-profonde-par-limmigration/
https://www.polemia.com/derriere-papy-biden-letat-profond-americain-a-la-manoeu

ERROR:root:Importer.get_webpage:https://www.wikistrike.com/2023/01/guerre-en-ukraine-la-russie-envoie-de-nouvelles-troupes-en-bielorussie-alors-que-les-craintes-d-une-nouvelle-attaque-grandissent.html


In [21]:
df_pages = pd.DataFrame(pages)

In [22]:
df_pages = df_pages[df_pages['html']!='']

In [23]:
df_pages['site']=df_pages['url'].apply(imp.get_site)

In [24]:
import patat.imp.selectors

In [25]:
df_pages[~df_pages['site'].isin(patat.imp.selectors.selectors)].value_counts('site')

Series([], dtype: int64)

## Développement et debugging parser
- lumieresurgaia.com                   4 OK
- planetes360.fr                       3 OK mais articles vides !
- www.la-petite-souris-normande.com    3 OK
- www.revue-elements.com               3 Problemes date, article vide sauf abstract
- bonsens.info                         2 OK
- elucid.media                         2 OK sans date
- extime.fr                            2 OK
- lecourrierdesstrateges.fr            2 OK
- qactus.fr                            2 OK
- www.epochtimes.fr                    2 OK

In [26]:
import patat.imp.selectors

In [27]:
importlib.reload(patat.imp.selectors)
importlib.reload(patat.imp.parser)
parser = patat.imp.parser.Parser()

In [28]:
site = 'lecourrier-du-soir.com'

In [29]:
df_site = df_pages[df_pages['site']==site]

In [30]:
for url in df_site['url']:
    print(url)

https://lecourrier-du-soir.com/coup-de-theatre-en-france-lansm-nexclut-pas-que-les-vaccins-pfizer-et-moderna-aient-provoque-la-surdite/
https://lecourrier-du-soir.com/macron-recase-ses-amis-il-promeut-castex-president-de-lagence-de-financement-des-infrastructures-de-transport-de-france/
https://lecourrier-du-soir.com/coup-de-tonnerre-en-espagne-le-gouvernement-des-baleares-avoue-que-16-personnes-ont-ete-tuees-par-le-vaccin/
https://lecourrier-du-soir.com/coup-de-theatre-une-revue-scientifique-mondiale-confirme-que-le-vaccin-provoque-des-troubles-menstruels/
https://lecourrier-du-soir.com/gros-scandale-en-france-en-pleine-crise-financiere-letat-va-emprunter-270-milliards-deuros-en-2023-du-jamais-vu/
https://lecourrier-du-soir.com/france-la-plainte-contre-le-professeur-perronne-rejetee-en-colere-lordre-des-medecins-menace-de-faire-appel/
https://lecourrier-du-soir.com/la-russie-crache-le-morceau-le-but-des-etats-unis-est-daffaiblir-leurope-militairement-et-de-la-desindustrialiser/
https:

In [31]:
df_parsed = pd.DataFrame(list(df_site.progress_apply(lambda row: parser.parse_article(row['url'],row['html'],row['site']), axis=1)))

  0%|          | 0/9 [00:00<?, ?it/s]

In [32]:
def list_count(obj):
    if type(obj) in [list,str]:
        return len(obj)
    else:
        return 0

In [33]:
df_pages['site'][392]

'www.mondialisation.ca'

In [34]:
df_parsed['c_count']=df_parsed['article'].apply(list_count)
df_parsed['p_count']=df_parsed['paragraphs'].apply(list_count)
df_parsed

,url,site,title,article,author,date,tags,paragraphs,date_iso,abstract,c_count,p_count
0,https://lecourrier-du-soir.com/coup-de-theatre...,lecourrier-du-soir.com,Coup de théâtre en France : l’ANSM n’exclut pa...,Dans un communiqué rendu public ce 30 juin et ...,By Cheikh Dieng -,1 juillet 2022,[InternationalEuropePolitiqueDiversSanté],"[« À ce stade, le lien de causalité entre ce v...",2022-07-01T07:26:23+00:00,,6569,24
1,https://lecourrier-du-soir.com/macron-recase-s...,lecourrier-du-soir.com,Macron recase ses amis : il promeut Castex pré...,"Après Matignon, Jean Castex pourrait prendre l...",By Cheikh Dieng -,8 juillet 2022,[InternationalEuropePolitique],"[Après Matignon, Jean Castex pourrait prendre ...",2022-07-08T11:10:04+00:00,,1003,6
2,https://lecourrier-du-soir.com/coup-de-tonnerr...,lecourrier-du-soir.com,Coup de tonnerre en Espagne : le gouvernement ...,"Accablées par la justice, les autorités sanita...",By Cheikh Dieng -,7 septembre 2022,[InternationalEuropePolitiqueDiversSanté],"[Accablées par la justice, les autorités sanit...",2022-09-07T09:49:57+00:00,,3358,11
3,https://lecourrier-du-soir.com/coup-de-theatre...,lecourrier-du-soir.com,Coup de théâtre : une revue scientifique mondi...,Une étude menée par le British Medical Journal...,By Cheikh Dieng -,28 septembre 2022,[InternationalAmérique du NordPolitiqueDiversS...,[Une étude menée par le British Medical Journa...,2022-09-28T09:32:55+00:00,,2670,10
4,https://lecourrier-du-soir.com/gros-scandale-e...,lecourrier-du-soir.com,Gros scandale en France : en pleine crise fina...,La France va emprunter 270 milliards d’euros e...,By Cheikh Dieng -,27 septembre 2022,[EconomieInternationalEurope],[La France va emprunter 270 milliards d’euros ...,2022-09-27T08:19:35+00:00,,3275,13
5,https://lecourrier-du-soir.com/france-la-plain...,lecourrier-du-soir.com,France : la plainte contre le professeur Perro...,L’Ordre des Médecins veut aller jusqu’au bout ...,By Cheikh Dieng -,25 octobre 2022,[InternationalEuropePolitiqueDiversSanté],[L’Ordre des Médecins veut aller jusqu’au bout...,2022-10-25T09:18:16+00:00,,3344,8
6,https://lecourrier-du-soir.com/la-russie-crach...,lecourrier-du-soir.com,La Russie crache le morceau : « le but des Eta...,La guerre en Ukraine est-elle l’opportunité ta...,By Cheikh Dieng -,31 octobre 2022,[InternationalAmérique du NordEuropePolitique],[La guerre en Ukraine est-elle l’opportunité t...,2022-10-31T09:09:49+00:00,,1920,5
7,https://lecourrier-du-soir.com/gros-scandale-a...,lecourrier-du-soir.com,Gros scandale aux USA : en seulement 9 mois de...,"En seulement neuf mois de guerre, les aides fi...",By Cheikh Dieng -,14 novembre 2022,[InternationalAmérique du NordPolitique],"[En seulement neuf mois de guerre, les aides f...",2022-11-14T09:33:06+00:00,,3042,11
8,https://lecourrier-du-soir.com/31556-2/,lecourrier-du-soir.com,France : les riches ont pris le pouvoir : 19 m...,Le gouvernement d’Elisabeth Borne compte 19 mi...,By Cheikh Dieng -,4 décembre 2022,[InternationalEuropePolitique],[Le gouvernement d’Elisabeth Borne compte 19 m...,2022-12-04T00:01:33+00:00,,2971,10


In [35]:
print(df_parsed['abstract'][1])

## Parsing de l'ensemble du Recueil

In [36]:
df_articles = pd.DataFrame(list(df_pages.progress_apply(lambda row: parser.parse_article(row['url'],row['html'],row['site']), axis=1)))



  0%|          | 0/619 [00:00<?, ?it/s]

In [37]:
df_articles['c_count']=df_articles['article'].apply(list_count)
df_articles['p_count']=df_articles['paragraphs'].apply(list_count)
df_articles

,url,site,title,article,author,date,tags,paragraphs,date_iso,abstract,c_count,p_count
0,https://www.alnas.fr/actualite/communaute/lara...,www.alnas.fr,L’Arabie saoudite organise des célébrations d'...,L’Arabie saoudite a organisé un événement pour...,Par Youcef -,31/10/2022,[ActualitésCommunauté],[L’Arabie saoudite a organisé un événement pou...,2022-10-31T00:00:00,,2277,8
1,https://www.alnas.fr/actualite/en-vrac/quel-ar...,www.alnas.fr,« Quel art ! » : un étudiant en droit invente ...,Une professeure de droit a partagé sur Twitter...,Par Oussama -,19/10/2022,[ActualitésEn vrac],[Une professeure de droit a partagé sur Twitte...,2022-10-19T00:00:00,,1683,8
2,https://www.alnas.fr/actualite/politique/laust...,www.alnas.fr,L’Australie ne reconnaît plus Jérusalem comme ...,Le gouvernement israélien a qualifié la positi...,Par Oussama -,18/10/2022,[ActualitésPolitique],[Le gouvernement israélien a qualifié la posit...,2022-10-18T00:00:00,,2678,15
3,https://www.alnas.fr/actualite/communaute/nant...,www.alnas.fr,Nantes : une mère musulmane meurt poignardée e...,NANTES – Une enquête pour « homicide volontair...,Par Sarah -,17/10/2022,[ActualitésCommunauté],[NANTES – Une enquête pour « homicide volontai...,2022-10-17T00:00:00,,1792,5
4,https://www.alnas.fr/actualite/en-vrac/la-fran...,www.alnas.fr,La France condamnée pour avoir placé un enfant...,La Cour européenne des droits de l’Homme (CEDH...,Par Sarah -,10/11/2022,[ActualitésEn vrac],[La Cour européenne des droits de l’Homme (CED...,2022-11-10T00:00:00,,1482,5
...,...,...,...,...,...,...,...,...,...,...,...,...
614,https://www.wikistrike.com/2022/10/selon-un-an...,www.wikistrike.com,"Selon un ancien conseiller du Pentagone, les É...","Selon un ancien conseiller du Pentagone, les c...",,Publié par wikistrike.com\n ...,[],"[Selon un ancien conseiller du Pentagone, les ...",2022-10-04T00:00:00,,3976,14
615,https://www.wikistrike.com/2022/10/la-pologne-...,www.wikistrike.com,La Pologne s’apprête à s’emparer du territoire...,La Pologne se prépare à s’emparer du territoir...,,Publié par wikistrike.com\n ...,[],[La Pologne se prépare à s’emparer du territoi...,2022-10-31T00:00:00,,2284,8
616,https://www.wikistrike.com/2022/11/exclusivite...,www.wikistrike.com,Exclusivité WikiStrike: un physicien en colère...,"Crise énergétique battant son plein, WikiStri...",,Publié par wikistrike.com\n ...,[],"[Crise énergétique battant son plein, WikiStr...",2022-11-19T00:00:00,,27223,27
617,https://www.wikistrike.com/2022/12/zelensky-pa...,www.wikistrike.com,Zelensky parti demander à papa Biden une aide ...,"Alors que des milliers d’Ukrainiens meurent, Z...",,Publié par wikistrike.com\n ...,[],"[Alors que des milliers d’Ukrainiens meurent, ...",2022-12-23T00:00:00,,557,5


In [38]:
df_articles[df_articles['c_count']==0]

,url,site,title,article,author,date,tags,paragraphs,date_iso,abstract,c_count,p_count
471,https://planetes360.fr/scoop-lancien-vice-pres...,planetes360.fr,None,None,,,[],[],,,0,0
472,https://planetes360.fr/exclu-360-le-professeur...,planetes360.fr,None,None,,,[],[],,,0,0


In [39]:
df_articles = df_articles[df_articles['c_count']!=0].copy()

In [40]:
import patat.ml.lex_analyser

In [41]:
importlib.reload(patat.ml.lex_analyser)

<module 'patat.ml.lex_analyser' from '/Volumes/Geek/Work/Patat/patat/ml/lex_analyser.py'>

In [42]:
lex = patat.ml.lex_analyser.LexAnalyser()

In [43]:
def get_parag(row):
    parags = row['paragraphs']
    if type(parags)==list:
        if len(parags)>0:
            return parags
    parags = row['article'].split('\n')
    print(parags)
    return parags

In [44]:
df_articles.loc[:,'paragraphs'] = df_articles.apply(get_parag,axis=1)

['Russie Ukraine, comment l’Amérique mène sa guerre en Europe 7,90€Ajouter au panier  Chez nous ! 25,50€Lire la suite']
["Le festival de l'Eurovision 2022 s'est tenu à Turin (Italie). Il est organisé par une association, l'Union européenne de radio-télévision (European Broadcasting Union), réunissant les télévisions de service public du continent européen devant 200 millions de téléspectateurs.", "la délégation ukrainienne à l’Eurovision 2022. L’un des accompagnateurs (à gauche) arbore le trident. Ce symbole médiéval ne devint l’emblème de l’Ukraine qu’en 1917-18 à l’initiative de l’historien Mykhailo Hrushevsky. Il a été adopté comme blason du pays en 1996. Il existe deux manières de dessiner sa base. Les variations de couleurs sont aussi significatives. Celle de l’accompagnateur n’est pas la version officielle, mais celle des bandéristes.À l'initiative de la Française Delphine Ernotte (présidente de l'association et de France-Télévision), la Russie, quoique faisant partie des 56 pays

In [45]:
df_articles[df_articles['p_count']==0]

,url,site,title,article,author,date,tags,paragraphs,date_iso,abstract,c_count,p_count
535,https://www.revue-elements.com/lukraine-ou-le-...,www.revue-elements.com,L’Ukraine ou le tombeau de l’Occident,"Russie Ukraine, comment l’Amérique mène sa gue...",Hervé Juvin,23 août 2022,[],"[Russie Ukraine, comment l’Amérique mène sa gu...",2022-08-23T00:00:00,Où en est-on du conflit Russie-États-Unis avec...,115,0
546,https://fr.sott.net/article/39998-Le-Pentag,fr.sott.net,Le Pentagone organise la victoire de l'Ukraine...,Le festival de l'Eurovision 2022 s'est tenu à ...,,,[],[Le festival de l'Eurovision 2022 s'est tenu à...,,,2732,0
547,https://fr.sott.net/article/40046-OTAN-quand-l...,fr.sott.net,OTAN : quand la Finlande déclare la guerre,"En compagnie de la Suède, la Finlande a fait a...",,,[],"[En compagnie de la Suède, la Finlande a fait ...",,,8557,0
548,https://fr.sott.net/article/40039-Effet-domino...,fr.sott.net,Effet domino de la reddition des soldats ukrai...,Alors que plus de 2400 soldats ukrainiens qui ...,,,[],[Alors que plus de 2400 soldats ukrainiens qui...,,,5565,0
549,https://fr.sott.net/article/40029-Guerre-en-Uk...,fr.sott.net,Guerre en Ukraine: les seize preuves de l'enti...,Afin de développer un politique de classe prol...,,,[],[Afin de développer un politique de classe pro...,,,14764,0
550,https://fr.sott.net/article/39999-Otan-apres-l...,fr.sott.net,"Otan : après la Finlande, la Suède va official...","Après celle de la Finlande dimanche, la Suède ...",,,[],"[Après celle de la Finlande dimanche, la Suède...",,,4137,0
551,https://fr.sott.net/article/40035-La-Chine-aug...,fr.sott.net,La Chine augmente ses importations d'énergie e...,"Les achats de pétrole, de gaz et de charbon on...",,,[],"[Les achats de pétrole, de gaz et de charbon o...",,,1919,0
552,https://fr.sott.net/article/40192-La-tyrannie-...,fr.sott.net,La tyrannie digitale et le code QR - prof. M...,« L'Organisation mondiale de la santé a commen...,,,[],[« L'Organisation mondiale de la santé a comme...,,,755,0
553,https://fr.sott.net/article/40291-L-etau-se-re...,fr.sott.net,L'étau se resserre autour de la Chine,Ce n'est pratiquement plus un secret pour pers...,,,[],[Ce n'est pratiquement plus un secret pour per...,,,6910,0
554,https://fr.sott.net/article/40371-L-OMS-recomm...,fr.sott.net,L'OMS recommande un nouveau vaccin contre la p...,"Le 13 novembre 2020, l'Organisation mondiale d...",,,[],"[Le 13 novembre 2020, l'Organisation mondiale ...",,,13002,0


In [46]:
df_articles['p_count']=df_articles['paragraphs'].apply(len)

# Sauvegarde des résultats
---

In [47]:
filename = 'data/tmp/230507-Recueil.csv'

In [48]:
df_articles.shape

(617, 12)

In [49]:
df_articles.to_csv(filename,index=False)

# Conclusions
---

- Parser sur tous les sites du recueil !

# Bricolages
---

In [ ]:
from bs4 import BeautifulSoup

In [70]:
url=df_articles['url'][602]

In [76]:
df_articles['article'][602].split('\n')

['Le 27 juillet 2022, l\'organisation "Children\'s Health Defense" présidée par Robert F. Kennedy a reçu les deux sondages qui avaient été commandés auprès de l\'institut d\'études de marché John Zogby Strategies, pour connaître les conséquences de la campagne de vaccination contre le Sars-CoV-2 au sein de la population américaine. Les résultats qui en découlent devraient conduire à des remises en question. Il a été constaté, lors de la seconde enquête, que 22 % des personnes de moins de 50 ans, qui n\'ont rien à craindre du Covid-19, ont été diagnostiquées avec une nouvelle maladie peu après la campagne de vaccination.',
 'Que nous enseigne la première enquête ?',
 "La première enquête, réalisée auprès de 1 038 participants, s'adressait aux adultes de tous âges. Des résultats inquiétants, si l’on en croit ses conclusions, puisque parmi l’ensemble des participants, 33 % n'étaient pas vaccinés, 67 % avaient été vaccinés au moins une fois et 15 % d’entre eux avaient déclaré avoir été dia

In [73]:
html = imp.get_webpage(url)

In [ ]:
soup = BeautifulSoup(html)

In [88]:
df = pd.read_csv('data/prod/Recueil.csv')

In [90]:
df['abstract'].fillna('',inplace=True)

In [92]:
df[df['abstract']!='']

,url,site,title,article,author,date,tags,paragraphs,date_iso,abstract,c_count,p_count
7,https://www.anguillesousroche.com/actualite/ou...,www.anguillesousroche.com,"Ouverture du gazoduc Baltic Pipe, qui relie la...","Les dirigeants de la Pologne, du Danemark et d...",NaN,28 septembre 2022,['Actualité / Europe'],['Le gazoduc a été inauguré lors d’une cérémon...,2022-09-28T00:00:00,"Les dirigeants de la Pologne, du Danemark et d...",2644,10
8,https://www.anguillesousroche.com/technologie/...,www.anguillesousroche.com,La CIA aurait investi dans une puissante techn...,L’adoption de cette technologie permettra aux ...,NaN,30 septembre 2022,['Etats-Unis / Génétique / Technologie'],['L’agence de renseignement américaine vient p...,2022-09-30T00:00:00,L’adoption de cette technologie permettra aux ...,5243,21
10,https://www.anguillesousroche.com/internet/rev...,www.anguillesousroche.com,Révélé : des documents montrent comment le DHS...,Encore des révélations sur la censure du gouve...,NaN,1 novembre 2022,['Actualité / Censure / Etats-Unis / Internet ...,['Un grand nombre de documents et de communica...,2022-11-01T00:00:00,Encore des révélations sur la censure du gouve...,8664,29
11,https://www.anguillesousroche.com/internet/rum...,www.anguillesousroche.com,Rumble déclare qu’il ne modifiera pas sa polit...,Rumble a quitté la France alors qu’elle cherch...,NaN,2 novembre 2022,['Actualité / Censure / Europe / France / Inte...,['La plateforme vidéo en pleine expansion Rumb...,2022-11-02T00:00:00,Rumble a quitté la France alors qu’elle cherch...,3072,14
12,https://www.anguillesousroche.com/science/ne-v...,www.anguillesousroche.com,"Ne vous inquiétez pas, nous ne manquerons jama...",Quand serons-nous à court de pétrole ? Dans 50...,NaN,10 novembre 2022,['Science / Société - Culture'],['Le pétrole brut est l’une des ressources les...,2022-11-10T00:00:00,Quand serons-nous à court de pétrole ? Dans 50...,41709,164
13,https://www.anguillesousroche.com/mysteres/10-...,www.anguillesousroche.com,10 raisons pour lesquelles le multivers pourra...,Notre Univers n’est peut-être pas le seul qui ...,NaN,20 novembre 2022,['Mystères'],['Pour simplifier les nombreuses définitions d...,2022-11-20T00:00:00,Notre Univers n’est peut-être pas le seul qui ...,9519,36
14,https://www.anguillesousroche.com/changement-c...,www.anguillesousroche.com,Article d’opinion du NY Times : « S’accoupler ...,« Nous serons plus nombreux à pouvoir monter d...,NaN,4 janvier 2023,['Changement climatique'],['Un article d’opinion du New York Times suggè...,2023-01-04T00:00:00,« Nous serons plus nombreux à pouvoir monter d...,3420,23
15,https://www.anguillesousroche.com/espace/pourq...,www.anguillesousroche.com,Pourquoi les trous noirs révèlent la majesté q...,Ce vide effrayant et tourbillonnant dont rien ...,NaN,Publié 31 mars 2023\n · Mis à j...,['Espace'],['Imaginez que vous trouviez une montre sur le...,2023-03-31T00:00:00,Ce vide effrayant et tourbillonnant dont rien ...,4481,9
78,https://www.dreuz.info/2022/04/attentats-crime...,www.dreuz.info,"Attentats, crimes, tueries et autres atrocités...",Dreuz publie chaque année depuis 9 ans la list...,Publié par Jean-Patrick Grumberg le 2 mai 2022,Publié par Jean-Patrick Grumberg le 2 mai 2022,[],['Nous ne sommes plus étonnés de constater qu’...,2022-05-02T00:00:00,Dreuz publie chaque année depuis 9 ans la list...,109600,2
79,https://www.dreuz.info/2022/05/le-proces-du-fa...,www.dreuz.info,Procès sur l’origine du dossier russe : le FBI...,Enquêteur spécial John Durham\nLe premier proc...,Publié par Jean-Patrick Grumberg le 28 mai 2022,Publié par Jean-Patrick Grumberg le 28 mai 2022,[],"[""En septembre 2021, après de longs mois qui m...",2022-05-28T00:00:00,1,49860,119
